###  Алгоритмы анализа данных (06 окт 2022 - 01 ноя 2022)
### Урок 1. Алгоритм линейной регрессии. Градиентный спуск.

# Домашнее задание
Сдайте задание до: 14 окт., 00:00 MSK

Загружаем необходимые библиотеки и функции

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [90]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

В качестве предмета исследования берем задачу из лекции

In [91]:
# Задача: предсказание баллов ЕГЭ ученика в зависимости от кол-ва лет стажа его репетитора
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

__1.__ Проведите небольшое исследование алгоритма градиентного спуска. Оцените влияние значений скорости обучения (eta) и количества итераций на ошибку алгоритма. Как связаны эти два гиперпараметра между собой? Подберите скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК. Как можно ускорить процесс вычисления весов?

__Решение 1__

Вычислим значение W_final и MSE для следующих значений скорости обучения (eta) и количества итераций (n_iter):

eta = 8e-3, 9e-3, 1e-2, 2e-2, 3e-2, 4e-2, 5e-2, 6e-2, 7e-2

n_iter = 100, 200, 300, 400

In [92]:
etas = [8e-3, 9e-3, 1e-2, 2e-2, 3e-2, 4e-2, 5e-2, 6e-2, 7e-2]
n_iters = [100, 200, 300, 400]

Подготовим функцию, которая будет возвращать Eta = {eta}, Iterations {n_iter}, W_final = {W}, MSE = {round(err, 3)}

In [93]:
def grad(eta, n_iter):
    
    temp_eta = eta

    for i in range(n_iter):        
        y_pred = np.dot(X, W)
        err = calc_mse(y, y_pred)
        for k in range(W.shape[0]):
            W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
        if i % 10 == 0: 
            temp_eta /= 1.1
    
    return f'Eta = {eta}, Iterations = {n_iter}: W_final = {W}, MSE = {round(err, 3)}'

Теперь организуем иттеративное исследование скорости сходимости алгоритма

In [94]:
W = np.array([1, 0.5])
print(f'Number of objects = {n}, \nInitial weights = {W} \n')

for eta in etas:
    print()
    for n_iter in n_iters:
        W = np.array([1, 0.5])
        print(grad(eta, n_iter))


Number of objects = 10, 
Initial weights = [1.  0.5] 


Eta = 0.008, Iterations = 100: W_final = [24.83058465  7.48124591], MSE = 239.604
Eta = 0.008, Iterations = 200: W_final = [35.33465138  5.57649536], MSE = 89.197
Eta = 0.008, Iterations = 300: W_final = [40.38518512  4.66065894], MSE = 54.425
Eta = 0.008, Iterations = 400: W_final = [42.81356754  4.22030923], MSE = 46.386

Eta = 0.009, Iterations = 100: W_final = [26.60723725  7.15907735], MSE = 207.053
Eta = 0.009, Iterations = 200: W_final = [36.96812593  5.28028994], MSE = 75.34
Eta = 0.009, Iterations = 300: W_final = [41.51235322  4.45626438], MSE = 50.004
Eta = 0.009, Iterations = 400: W_final = [43.50542566  4.09485142], MSE = 45.13

Eta = 0.01, Iterations = 100: W_final = [28.22929764  6.86494171], MSE = 179.895
Eta = 0.01, Iterations = 200: W_final = [38.32845066  5.03361602], MSE = 65.722
Eta = 0.01, Iterations = 300: W_final = [42.36857287  4.30100215], MSE = 47.45
Eta = 0.01, Iterations = 400: W_final = [43.98480618  

Результаты показывают, что с ростом начального значения eta ошибка алгоритма сокращается быстрее.

Но это только до определенного уровня, а вот когда eta превышает значение 0.06 алгоритм идет вразнос

__Оптимальное соотношение для получения достаточно близкого к МНК результата будут значения: eta=0.05 , n_iter=300__

__Процесс вычисления весов можно ускорить повышая начальное значение eta (до определенного уровня) и меняя начальное значение весов.__  

__2*.__ В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [ ]:
w = np.array([1, 0.5])
for i in range(1001):
    y_pred = np.dot(w, X.T)
    err = calc_mse(y, y_pred)
    w -= (eta * (1/n * 2 * np.sum(X.T * (y_pred - y)))) # ошибка!
    if i % 100 == 0:
        print(i, w, err)

__Решение 2__

Воспользуемся формулами из лекции:

$$\nabla_{w}Q(w,X) = \frac{2}{l}X^{T}(Xw-y)$$

$$w^{k} = w^{k-1} - \eta_{k}\nabla Q(w^{k-1}, X) = w^{k-1} - \eta_{k}\frac{2}{l}X^{T}(Xw-y)$$

из этих формул видно, что в предложенном коде есть 2 ошибки:

__1 ошибка__ в 3 строке: $y_pred = np.dot(w, X.T)$ 

выражние $np.dot(w, X.T)$ надо поменять на $np.dot(X, w)$

__2  ошибка__ в 5 строке: $W -= (eta * (1/n * 2 * np.sum(X.T * (y_pred - y))))$ 

выражние $np.sum(X.T * (y_pred - y))$ надо поменять на $X.T @ (y_pred - y)$

In [95]:
n = X.shape[0]

eta = 5e-2 
n_iter = 300

w = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {w} \n')

for i in range(n_iter):
    # ИЗМЕНЕНИЯ выражние np.dot(w, X.T) надо поменять на np.dot(X, w)
    y_pred = np.dot(X, w)
    # конец ИЗМЕНЕНИЙ
    err = calc_mse(y, y_pred)

    # ИЗМЕНЕНИЯ в выражении w -= (eta * (1/n * 2 * np.sum(X.T * (y_pred - y))))
    # выражние np.sum(X.T * (y_pred - y)) надо поменять на X.T @ (y_pred - y)
    w -= eta * (1/n * 2 * X.T @ (y_pred - y))
    # конец ИЗМЕНЕНИЙ
    
    if i % 30 == 0:       
        print(f'Iteration #{i}: W_new = {w}, MSE = {round(err,5)}')

Number of objects = 10        
Learning rate = 0.05        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [ 6.4  19.35], MSE = 3047.75
Iteration #30: W_new = [35.16120834  5.60976141], MSE = 94.66355
Iteration #60: W_new = [42.62118483  4.25519525], MSE = 47.05051
Iteration #90: W_new = [44.46057568  3.9216497 ], MSE = 44.15609
Iteration #120: W_new = [44.91409111  3.83941166], MSE = 43.98014
Iteration #150: W_new = [45.02590869  3.81913527], MSE = 43.96944
Iteration #180: W_new = [45.05347814  3.81413597], MSE = 43.96879
Iteration #210: W_new = [45.06027559  3.81290336], MSE = 43.96875
Iteration #240: W_new = [45.06195156  3.81259945], MSE = 43.96875
Iteration #270: W_new = [45.06236478  3.81252452], MSE = 43.96875


__3*.__ Вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию начинает изменяться ниже определенного порога. Сколько нужно сделать итераций, если установить допустимое отклонение mse в размере diff=1e-6, а значение eta=1e-2?

__Решение 3__

Введем другие условия останова алгоритма - когда веса перестают изменяться меньше порога min_weight_dist или когда количество итераций превышает n_iter (последнее условие, выходит за рамки задания, но его таки разумно сделать, чтобы подстраховаться от ухода цикла в небеса)

Для этого поменяем цикл for на цикл while и организуем расчет разницы weight_dist и количества иттераций iter_num

In [98]:
n = X.shape[0]

eta = 1e-2 
n_iter = 3000

# ИЗМЕНЕНИЯ добавим счетчик итераций
iter_num = 0

# ИЗМЕНЕНИЯ добавим критерий сходимости (разница весов, при которой алгоритм останавливается)
min_weight_dist = 1e-6

# ИЗМЕНЕНИЯ зададим начальную разницу весов числом превышающим min_weight_dist
weight_dist = 1
# конец ИЗМЕНЕНИЙ

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

# ИЗМЕНЕНИЯ меняем конструкцию цикла
#for i in range(n_iter):
while weight_dist > min_weight_dist and iter_num < n_iter:
# конец ИЗМЕНЕНИЙ
    
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    
    # ИЗМЕНЕНИЯ организуем расчет разницы weight_dist
    new_W = W - eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    weight_dist = np.linalg.norm(new_W - W, ord=2)
    # конец ИЗМЕНЕНИЙ

    if iter_num % 100 == 0:
        #eta /= 1.1
        print(f'Iter #{iter_num}: W_new = {new_W}, MSE = {round(err,2)}, weight_dist = {weight_dist}')
     
    # ИЗМЕНЕНИЯ
    iter_num += 1
    W = new_W
    # конец ИЗМЕНЕНИЙ

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iter #0: W_new = [2.08 4.27], MSE = 3047.75, weight_dist = 3.921645062980585
Iter #100: W_new = [28.38281518  6.83710367], MSE = 177.43, weight_dist = 0.15602112986598138
Iter #200: W_new = [38.38986469  5.02247953], MSE = 65.33, weight_dist = 0.06241557391934305
Iter #300: W_new = [42.39314129  4.29654705], MSE = 47.39, weight_dist = 0.024969078682016872
Iter #400: W_new = [43.99463466  4.00614091], MSE = 44.52, weight_dist = 0.00998877124856895
Iter #500: W_new = [44.63530512  3.8899652 ], MSE = 44.06, weight_dist = 0.0039959644617626805
Iter #600: W_new = [44.89160255  3.84348962], MSE = 43.98, weight_dist = 0.0015985681904560113
Iter #700: W_new = [44.99413322  3.82489726], MSE = 43.97, weight_dist = 0.0006395002468072415
Iter #800: W_new = [45.03515017  3.81745947], MSE = 43.97, weight_dist = 0.0002558292903013498
Iter #900: W_new = [45.05155882  3.81448401], MSE = 43.97, weight_dist = 0.0001

__Если установить допустимое отклонение mse в размере diff=1e-6, а значение eta=1e-2, то для получения ответа нужно сделать  1400 итераций__  

Для сравнения: Если установить допустимое отклонение mse в размере diff=1e-6, а значение __eta=5e-2__ для получения ответа нужно сделать всего __300__ итераций